In [36]:
import pandas as pd
import numpy as np

import nltk 
import gensim

import sys, os, datetime, re, string

In [16]:
ROOT = "C:\\Users\\Evan\\MyFile\\School Thing\\PHBS_ai_for_fintech\\PHBS_AI_TweetSentimentExtraction"
sys.path.append(ROOT)
sys.path

['C:\\Users\\Evan\\MyFile\\School Thing\\PHBS_ai_for_fintech\\PHBS_AI_TweetSentimentExtraction\\02 src\\01 dataCleaning\\teamEvan\\quickTry',
 'C:\\Users\\Evan\\.conda\\envs\\env_nlp\\python37.zip',
 'C:\\Users\\Evan\\.conda\\envs\\env_nlp\\DLLs',
 'C:\\Users\\Evan\\.conda\\envs\\env_nlp\\lib',
 'C:\\Users\\Evan\\.conda\\envs\\env_nlp',
 '',
 'C:\\Users\\Evan\\.conda\\envs\\env_nlp\\lib\\site-packages',
 'C:\\Users\\Evan\\.conda\\envs\\env_nlp\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Evan\\.ipython',
 '../../../..',
 '../../../..',
 'C:\\Users\\Evan\\MyFile\\School Thing\\PHBS_ai_for_fintech\\PHBS_AI_TweetSentimentExtraction']

In [38]:
train = pd.read_csv(os.path.join(ROOT, '01 data/train.csv'))
test = pd.read_csv(os.path.join(ROOT, '01 data/test.csv'))
ss = pd.read_csv(os.path.join(ROOT, '01 data/sample_submission.csv'))


train = train.dropna() #drop one nan
train.info()
test.info()

train.describe()

train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27480 entries, 0 to 27480
Data columns (total 4 columns):
textID           27480 non-null object
text             27480 non-null object
selected_text    27480 non-null object
sentiment        27480 non-null object
dtypes: object(4)
memory usage: 1.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
textID       3534 non-null object
text         3534 non-null object
sentiment    3534 non-null object
dtypes: object(3)
memory usage: 82.9+ KB


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [30]:
countDistribution = train.groupby('sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
countDistribution

,sentiment,text
1,neutral,11117
2,positive,8582
0,negative,7781


In [98]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
#     髒話
    text = re.sub('\*{2,}', 'MYCURSE', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
#     <g86g86>
    text = re.sub('<.*?>+', '', text) 
#     [g86g86]
    text = re.sub('\[.*?\]', '', text)
#     標點
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
#     數字
    text = re.sub('\w*\d\w*', 'MYNUM', text)
    return text


#  想一下

In [107]:
def find_re(text):
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\*{2,}', 'MYCURSE', text)
    aa = re.findall('\*.*?\*', text)
#
    
    return aa

aa = train['text'].apply(lambda x:find_re(x))
for i, a in aa.items():
    if len(a)>0:
        print(i, a)
        print(train['text'][i])
        

16 ['*sigh*']
 S`ok, trying to plot alternatives as we speak *sigh*
92 ['*just joined twitter...*']
 Hi  how are you doing ???  *just joined twitter...*
192 ['*phew*']
 *phew*  Will make a note in case anyone else runs into the same issueï¿½
203 ['*huggles*']
 more nightmares?  *huggles*
277 ['*laughs*']
 *laughs* I`m glad that you have self confidence - it`s a wonderful trait to have  I`ll applaud extra loud for it, okay?
705 ['*points at the gear question I just posted*']
 *points at the gear question I just posted* I cant get the rest of my Dreadweave set
805 ['*tears*']
 how sad! are you saying that I`m fat?  *tears*
963 ['*|)|/-\\|\\|/-\\*']
mad the rain got me...now i cant go see jaiden   *|)|/-\|\|/-\*
993 ['*LOVE*']
 I see you didn`t shave your head. But I *LOVE* the cut.  I`m glad you didn`t shave it. Your hair`s too pretty!
1057 ['*sigh*']
 oh dear.. u serious? so how does one prevent bites??  I`m so itchy its distracting while editing.. *sigh*
1153 ['*hugs*']
_tah oh no.  *h

In [101]:
train['cleanedText']  = train['text'].apply(lambda x:clean_text(x))
# train['selected_text'] = train['selected_text'].apply(lambda x:clean_text(x))

train.loc[ :, ['text', 'cleanedText']].head(300)

,text,cleanedText
0,"I`d have responded, if I were going",id have responded if i were going
1,Sooo SAD I will miss you here in San Diego!!!,sooo sad i will miss you here in san diego
2,my boss is bullying me...,my boss is bullying me
3,what interview! leave me alone,what interview leave me alone
4,"Sons of ****, why couldn`t they put them on t...",sons of MYCURSE why couldnt they put them on ...
5,http://www.dothebouncy.com/smf - some shameles...,some shameless plugging for the best rangers...
6,2am feedings for the baby are fun when he is a...,MYNUM feedings for the baby are fun when he is...
7,Soooo high,soooo high
8,Both of you,both of you
9,Journey!? Wow... u just became cooler. hehe....,journey wow u just became cooler hehe is tha...
